# Package List

In [21]:
import numpy as np, pandas as pd, warnings, os, openai, json
from tqdm.auto import tqdm
from openai import OpenAI
warnings.filterwarnings('ignore')

# Data Load

In [2]:
df = pd.read_csv("../data/서울시대여공구찾기정보.csv", encoding='cp949')

## Preprocessing

In [3]:
del_col = df.isnull().sum()[df.isnull().sum()/df.shape[0]>0.95].index.tolist()
df = df[[x for x in df.columns if x not in del_col]]
df['생성일시'] = pd.to_datetime(df['생성일시'], format='%Y%m%d%H%M%S')
del df['위치(위도,경도)']
del df['요금']

In [4]:
df['요금(과금기준 + 요금)'].fillna('0원', inplace=True)
def split_period_and_price(x):
    if '1일' in x:
        return pd.Series(['1일', x.replace('1일', '').strip()])
    elif '1시간' in x:
        return pd.Series(['1시간', x.replace('1시간', '').strip()])
    else:
        return pd.Series(['없음', x.strip()])

df[['과금기준', '요금']] = df['요금(과금기준 + 요금)'].apply(split_period_and_price)
del df['요금(과금기준 + 요금)']

In [28]:
user_info = df.T.to_dict()

# Location Info 

In [5]:
import requests

def get_lat_lon(location_name):
    url = 'https://nominatim.openstreetmap.org/search'
    params = {
        'q': location_name,
        'format': 'json'
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; ChatGPT-Example/1.0; +http://yourdomain.com)'
    }

    response = requests.get(url, params=params, headers=headers)

    # 응답 확인
    if response.status_code != 200:
        print(f"요청 실패: {response.status_code}")
        print("응답 내용:", response.text)
        return None

    try:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return float(lat), float(lon)
        else:
            print("위치 정보를 찾을 수 없습니다.")
            return None
    except ValueError as e:
        print("JSON 파싱 오류:", e)
        print("응답 내용:", response.text)
        return None

# 테스트
location = '강남역'
result = get_lat_lon(location)
print(f"{location} 위도/경도: {result}")


강남역 위도/경도: (37.4999282, 127.0263182)


In [ ]:
key = open('../../../api_key.txt','r')
api_key = key.read()
openai.api_key = api_key

In [29]:
client = OpenAI(api_key=api_key)
def invoke(question, info = user_info, model="gpt-4o", temperature=0.7):
    response = client.chat.completions.create(
        model=model,
        messages = [
                    {"role": "system", 
                        "content": f"""너는 사람들의 질문에 친절히 답해주는 도우미야.
                        너는 서울시에서 운영하는 대여 공구 찾기 정보 시스템이야. 
                        관련 정보는 {user_info}와 같아. 
                        이정보를 통해서 사용자의 답변에 친절해 답해줘.
                        """
                        },
                    {"role": "user", 
                    "content": question}
                ],
        temperature=temperature
    )
    return response.choices[0].message.content

In [35]:
[x for x in dir(np) if 'log' in x]

['log',
 'log10',
 'log1p',
 'log2',
 'logaddexp',
 'logaddexp2',
 'logical_and',
 'logical_not',
 'logical_or',
 'logical_xor',
 'logspace']

In [31]:
user_info

{0: {'구분자(PK)': 13423,
  '공구 대분류 코드': 4,
  '공구 대분류': '기타공구',
  '공구 중분류 코드': 44,
  '공구 중분류': '기타공구',
  '공구 이름': ' 고압세척기 ',
  '과금기준': '없음',
  '수량': 1,
  '장소 PK': 501,
  '대여장소명': '황학동 공구대여소 ',
  '상세주소': '서울특별시 중구 황학동 1010',
  '위도': 37.56742,
  '경도': 127.02137,
  '전화번호': '02-3396-6909',
  '평일오픈시간': 1000.0,
  '평일클로즈시간': 1700.0,
  '생성일시': Timestamp('2022-06-14 11:29:16'),
  '요금': '무료'},
 1: {'구분자(PK)': 14913,
  '공구 대분류 코드': 4,
  '공구 대분류': '기타공구',
  '공구 중분류 코드': 44,
  '공구 중분류': '기타공구',
  '공구 이름': '(57)레이저거리측정기',
  '과금기준': '없음',
  '수량': 1,
  '장소 PK': 172,
  '대여장소명': '우리마을 공구대여소(사당1동)',
  '상세주소': '서울 동작구 동작대로17길 28',
  '위도': 37.4830324,
  '경도': 126.9765661,
  '전화번호': '02-820-2571',
  '평일오픈시간': 900.0,
  '평일클로즈시간': 1800.0,
  '생성일시': Timestamp('2022-09-21 15:47:07'),
  '요금': '무료'},
 2: {'구분자(PK)': 14038,
  '공구 대분류 코드': 4,
  '공구 대분류': '기타공구',
  '공구 중분류 코드': 44,
  '공구 중분류': '기타공구',
  '공구 이름': '2단 사다리',
  '과금기준': '없음',
  '수량': 1,
  '장소 PK': 13214,
  '대여장소명': '묵1동 공유마을 공유대여소',
  '상세주소': '공릉로2가길 5',
  

In [32]:
question = f"나는 고압세척기를 빌리고싶어. 내가 사는 곳은 황학동이야. 근처에 있는 대여 장소가 어디 있는지 알려줘"

In [33]:
answer = invoke(question)
print("💬 GPT 응답:", answer)

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 128000 tokens. However, your messages resulted in 2656846 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [13]:
def invoke(question: str):
    prompt = f"""
    너는 도우미야 사용자의 물음에 친절히 답해줘야해.
    """
    response = client.responses.create(
        model='gpt-4.1',
        messages=[
                {"role": "system", "content": "당신은 완벽한 도우미입니다."},
                {"role": "user", "content": prompt}
            ],
        input = question
        )
    return response

In [14]:
output = invoke('배가 고픈데 생선을 먹고싶어. 6월에 먹을 수 있는 제철 생선 있어?')

TypeError: Responses.create() got an unexpected keyword argument 'messages'

In [20]:
def invoke(question: str):
    prompt = f"""
    너는 도우미야 사용자의 물음에 친절히 답해줘야돼
    """
    response = openai.ChatCompletion.create(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": "당신은 완벽한 도우미입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    result_text = response['choices'][0]['message']['content']

    try:
        result_dict = json.loads(result_text)
        with open("clustered_equipment.json", "w", encoding="utf-8") as f:
            json.dump(result_dict, f, ensure_ascii=False, indent=2)
        print("✅ 클러스터링 결과 저장 완료: clustered_equipment.json")
        return result_dict
    except json.JSONDecodeError:
        print("⚠️ JSON 형식 오류. 출력 확인 필요:")
        print(result_text)
        return None
    

In [19]:
question = '나 너무 배고파'
result = invoke(question)

# 만약 JSON 결과를 바로 보고 싶으면
import pandas as pd

# if result:
#     df_result = pd.DataFrame(result)
#     display(df_result)

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
prompt = f"""
    너는 도구가 어디있는지 찾아주는 도우미야. 
    사용자가 필요한 도구와 현재 위치를 알려주거야.
    너는 {df}를 참고해서 **사용자가 필요로 하는 도구**와 **현재 위치**에서 가장 가까이에있는 대여소를 찾아주면 돼.
    ============================
    """